<a href="https://colab.research.google.com/github/kowalski-k/projekt_TOM/blob/master/Projekt_TOM_U_NET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importowanie używanych bibliotek

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

#sprawdzenie używanej wersji tensorflow
print(tf.__version__)

# Pobranie danych i zapoznanie się z nimi

In [0]:
#pobieranie danych
! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
! sudo apt-get install git-lfs
! git lfs install
! git clone https://github.com/neheller/kits19.git 
%cd kits19
! python -m starter_code.get_imaging
!git checkout feature/google-colab
!make

In [0]:
#wczytanie danych dla jednego wybranego pacjenta
from starter_code.utils import load_case
volume, segmentation = load_case("case_00113")

In [0]:
#wizualizacja nowotworów i nerek na tle obrazu CT (zobaczyć jedynie jak działa, przyda się na końcu)
'''
visualize("case_00113", "case_113")
''' 

In [0]:
#wyodrębnienie i obrazowanie segementation
segmentation = segmentation.get_fdata()
(a,x,y) = segmentation.shape
for i in range(0,a,1):
  plt.figure()
  plt.imshow(segmentation[i],'gray')
  plt.title(i)

In [0]:
#wyodrębnienie i wczytywanie "czystych" obrazow bez zaznaczen (volume)
volume = volume.get_fdata()
(a,x,y) = volume.shape
for i in range(0,a,1):
  plt.figure()
  plt.imshow(volume[i],'gray')
  plt.title(i)

In [0]:
#zdobycie przydatnych w późniejszej części informacji o segmentation
i = 19
print(segmentation.shape)
print(type(segmentation[i]))
print(np.amax(segmentation[i]))
print(np.amin(segmentation[i]))
print(segmentation[i])
plt.imshow(segmentation[i],'gray')

In [0]:
#zdobycie przydatnych w późniejszej części informacji o volume
print(volume.shape)
print(type(volume[0]))
print(np.amax(volume[0]))
print(np.amin(volume[0]))
print(volume[0])
plt.imshow(volume[0],'gray')

# Przygotowanie danych do trenowania


In [0]:
#pierwsze 210 pacjentow ma posegmentowane nerki i nowotwory. Do trenowania uzyjemy jedynie 5 case'ów ze względu na to,
#że przy próbie wczytania większej ilości występuje błąd związany z brakiem odpowiedniej ilości pamięci RAM

all_volumes = []
all_segmentations = []

for i in range(5):
  volume, segmentation = load_case(i+5)
  segmentation = segmentation.get_fdata()
  volume = volume.get_fdata()
  all_volumes.append(volume)
  all_segmentations.append(segmentation)

In [0]:
#wczytanie obrazow bez oznaczen do jednej listy
volumes = []

for one_volume in all_volumes:
  (a,y,x) = one_volume.shape
  for i in range(0,a,1):
    volumes.append(one_volume[i])

In [0]:
#wczytanie masek dla nowotworow do jednej listy 

kidney = []
tumor = []

numbers_of_slice_kidney = []
numbers_of_slice_tumor = []



for one_segmentation in all_segmentations:
  (a,y,x) = one_segmentation.shape
  for i in range(0,a,1):
    for y_col in range(0,y,1):
      for x_row in range(0,x,1):
        if (one_segmentation[i][y_col][x_row] == 2):
          tumor.append(one_segmentation[i])
          numbers_of_slice_tumor.append(i)
          break
      else:
        continue
      break

In [0]:
#wczytanie masek dla nerek do jednej

for one_segmentation in all_segmentations:
  (a,y,x) = one_segmentation.shape
  for i in range(0,a,1):
    for y_col in range(0,y,1):
      for x_row in range(0,x,1):
        if (one_segmentation[i][y_col][x_row] == 1):
          kidney.append(one_segmentation[i])
          numbers_of_slice_kidney.append(i)
          break
      else:
        continue
      break

In [0]:
#wyznaczenie masek tylko dla wartosci nerek albo tylko wartosci nowotworow
#usunieto wartosci dla nerki w masce nowotworow i odwrotnie dla maski nerek 

for i in range(len(kidney)):
  for y_col in range(len(kidney[i])):
    for x_row in range(len(kidney[i][y_col])):
      if (kidney[i][y_col][x_row] == 2):
        kidney[i][y_col][x_row] = 0


for i in range(len(tumor)):
  for y_col in range(len(tumor[i])):
    for x_row in range(len(tumor[i][y_col])):
      if (tumor[i][y_col][x_row] == 1):
        tumor[i][y_col][x_row] = 0
      elif (tumor[i][y_col][x_row] == 2):
        tumor[i][y_col][x_row] = 1

In [0]:
#przygotowanie macierzy na trening
X_train = []
Y_train = []

#uzupełenie macierzy obrazami + zamiana rozmiaru obrazów z (512,512) na (512,512,1)
for i in range(len(kidney)):
  #volume
  img_volume = volumes[numbers_of_slice_kidney[i]]
  img_volume_new_shape = np.expand_dims(img_volume,axis=-1)
  X_train.append(img_volume_new_shape)
  #kidney
  img_kindey = kidney[i]
  img_kidney_new_shape = np.expand_dims(img_kindey,axis=-1)
  Y_train.append(img_kidney_new_shape)

#bez normalizacji
X_train = np.asarray(X_train)

#wybrane przekroje (volume) należy znormalizować do wartości [0,1]
for i in range(len(X_train)):
  X_train[i] = (X_train[i]-np.amin(X_train[i]))/(np.amax(X_train[i])-np.amin(X_train[i]))

#finalne przygotowane macierze do treningu
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

#informacje
print(X_train.shape)
print(Y_train.shape)
print(numbers_of_slice_kidney)
print(numbers_of_slice_tumor)